In [ ]:
# import packages, load function and connect to google drive
import plac
from tqdm import tqdm
import numpy as np
from PIL import Image
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pdb; 
import math
from math import nan

import sys
if 'google.colab' in sys.modules and 'torch' not in sys.modules:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

%matplotlib inline
import os

# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive', force_remount=True)

# %cd "/content/drive/My Drive/Colab Notebooks/Linear_approx"

# !pip install import-ipynb
# import import_ipynb
# import importlib

# import functions
# importlib.reload(functions)



In [ ]:
class Generator_d(nn.Module):

  def __init__(self, net_param):

    super(Generator_d, self).__init__()
    self.f = []
    for i in range(1,len(net_param)):
      net_param[i-1]
      layer = nn.Linear(net_param[i-1], net_param[i], bias = False)
      nn.init.normal_(list(layer.parameters())[0].data, 0, 1/np.sqrt(net_param[i]))
      self.f = np.append(self.f, [layer])
    
  def decoder(self, z):
    for i in range(len(self.f)):
      z = F.relu(self.f[i](z))
    return z 
  
  def forward(self, z):
    return self.decoder(z)

In [ ]:
class algo_parameter:
  def __init__(self, stepsize, max_itt, tolerance):
    self.stepsize = stepsize
    self.max_itt = max_itt
    self.tolerance = tolerance
    
class weight_mats():
  W = {} # create a data struct for weight matrices
  def __init__(self, net_param):
    for i in range(len(net_param)-1):
      self.W[i] =torch.randn(net_param[i+1],net_param[i])

In [ ]:
## function that solves quasi-gradient algorithm with successive error as stopping criteria
def quasi_grad(net_param, algo_param, generator, y0, Wm, A, out_toggle):
  x_dim = net_param[len(net_param)-1]
  z_dim = net_param[0]
  [m,_] = A.size()
  scale = 1 # this will change if the weight matrices are not normalized


  zk = torch.randn(1, z_dim).cuda()
  z_temp = zk
  
  itt = 0
  succ_error = 1
  while itt < algo_param.max_itt and succ_error > algo_param.tolerance:
    z_temp = zk - torch.t(algo_param.stepsize *  torch.matmul(torch.t(torch.matmul(A,Wm)), torch.matmul(A, generator(zk).reshape(x_dim)) -  y0.reshape(m)) * (scale**(-1)))
    itt += 1
    succ_error = torch.norm(z_temp-zk)/torch.norm(zk)
    zk.data = z_temp.data
    # pdb.set_trace()

    
    if out_toggle != 0:
      if itt % ((out_toggle // 10)*10 +1) == 0:
        print('====> In quasi-gradient: Iteration: {} Successive error: {:.4e}'.format(itt, succ_error))
  if out_toggle != 0:
    print('====> In quasi-gradient: Iteration: {} Successive error: {:.4e}'.format(itt, succ_error))  
    print('')

  return zk, itt

In [ ]:
tot_trials = 20
depth = 3
z_dim_list = range(1,20,1)
x_dim_list = range(1,442,20)
error_matrix_quasi = torch.zeros(len(z_dim_list), len(x_dim_list), tot_trials)
quasi_algo_param = algo_parameter(3, 10000, 1e-13) # (stepsize, max_itt, tolerance)
print(torch.cuda.memory_summary(device=None, abbreviated=False))
noise_level = 0

for j in range(len(z_dim_list)):
  z_dim = z_dim_list[j]
  for k in range(0, len(x_dim_list)):
    x_dim = x_dim_list[k]
    net_param = [z_dim, x_dim, x_dim, x_dim]    
    print('====> network parameter = {}'.format(net_param))
    for trials in range(tot_trials):  
      # print('====> trial number = {}'.format(trials))

      generator = Generator_d(net_param)
      # normalized signal
      z0 = torch.randn(net_param[0]).cuda()
      z0 = z0 /torch.norm(z0)


      for i in range(0, depth):
        generator.f[i].cuda()
        generator.f[i].requires_grad = False

      Wm = list(generator.f[0].parameters())[0].data
      for i in range(1, depth):
        Wm = torch.matmul(list(generator.f[i].parameters())[0].data,Wm)

      A = torch.eye(x_dim).cuda()

      # measurement for denoising
      y0 = generator(z0);

      z_quasi_est, itt = quasi_grad(net_param, quasi_algo_param, generator, y0, Wm , A, 0)
      error_matrix_quasi[j,k, trials] = torch.norm(generator(z0).detach()-generator(z_quasi_est).detach())/ torch.norm(generator(z0).detach())
      
      del generator.f
      del generator
      del Wm
      del A
      del y0

      del z0
      del z_quasi_est

      torch.cuda.empty_cache()
      gc.collect()
  
    del net_param
    gc.collect()
  gc.collect()
  # print(torch.cuda.memory_summary(device=None, abbreviated=False))

# save result
# torch.save(error_matrix_quasi,"/content/drive/My Drive/Colab Notebooks/Linear_approx/Results/Expansive_compare/Expansive_noiseless_phase.pt")

In [ ]:
torch.save(error_matrix_quasi,"/content/drive/My Drive/Colab Notebooks/Linear_approx/Results/Expansive_compare/Expansive_noiseless_phase.pt")
error_matrix_quasi = torch.load('/content/drive/My Drive/Colab Notebooks/Linear_approx/Results/Expansive_compare/Expansive_noiseless_phase.pt')
error_matrix_total = torch.zeros(len(x_dim_list), len(z_dim_list))
for j in range(len(z_dim_list)):
  for k in range(len(x_dim_list)):
    for trials in range(tot_trials):  
      if error_matrix_quasi[j, k, trials] < 1e-5:
        error_matrix_total[ len(x_dim_list) - 1 - k, j] = error_matrix_total[len(x_dim_list) - 1 -  k, j] + 1
fig, ax = plt.subplots(1)
plt.imshow(torch.kron(error_matrix_total/tot_trials, torch.ones(1,1)), cmap = 'gray')
ax.set_xticks(np.arange(0,26,2))
ax.set_xticklabels(['1','3','5','7','9','11','13','15','17','19'])
ax.set_yticks(np.arange(0,26,2))
ax.set_yticklabels(np.arange(441,0,-40))
plt.xlabel('code dimension, $n_0$')
plt.ylabel('layer widths with $n_1 = n_2 = n_3$')
plt.plot(range(0,19), 22 - 20*np.multiply(z_dim_list,1)/20)
fig.savefig("/content/drive/My Drive/Colab Notebooks/Linear_approx/Results/Phase_plot_non_expanse.png", dpi = 300)
